### character-level LSTM in Pytorch

#### 该笔记记录的模型如下所示
<img src="assets/charseq.jpeg" width="500">

#### 导入相关的包

In [1]:
import numpy as np 
import torch 
from torch import nn
import torch.nn.functional as F

####  读取数据

In [2]:
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

#### 对其进行标记（tokenization）
即是为其数据建造一个词典，让每个字符均有对应的标签

In [6]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch:li for li,ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [7]:
print(encoded[:100])

[14 45 48 54 76 72 74 46 75 22 22 22 82 48 54 54 18 46 80 48 12 49 27 49
 72 62 46 48 74 72 46 48 27 27 46 48 27 49 28 72  4 46 72  2 72 74 18 46
 70 39 45 48 54 54 18 46 80 48 12 49 27 18 46 49 62 46 70 39 45 48 54 54
 18 46 49 39 46 49 76 62 46  9 51 39 22 51 48 18  0 22 22 58  2 72 74 18
 76 45 49 39]


#### 将其转换为one-hot编码
构造一个函数，使其具有使得原本的文本转化为one-hot功能

In [15]:
def one_hot_encoded(arr,n_labels):
    
    one_hot = np.zeros((np.multiply(*arr.shape),n_labels),dtype = np.float32)
    
    one_hot [np.arange(one_hot.shape[0]),arr.flatten()] = 1
    
    ont_hot = one_hot.reshape((*arr.shape,n_labels))
    
    return one_hot

In [16]:
test_seq = np.array([[2,4,5,7,2]])
one_hot = one_hot_encoded(test_seq , 10)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


#### 建造一个mini-batch
此处对应的是英文文本的batch，且训练数据是一段英文文本，故其batch的组成是其不同的英文序列
且为了保留batch的完整，需要丢弃掉最后一部分空余的数据

In [ ]:
def get_batches(arr,batch_size,seq_length):
    
    batch_size_total = batch_size * seq_length
    
    n_batches = len(arr) //batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    
    arr = arr.reshape((batch_size , -1))
    
    for i in range(0,arr.shape[1],seq_length):
    

In [9]:
np.multiply?